In [1]:
# Mount google drive
from google.colab import drive
import gc

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import openai
from openai import OpenAI

import json
import os
import io
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import time
import datetime
from random import random

In [3]:
%cd ./drive/My Drive/SUNY_Poly_DSA598/

/content/drive/My Drive/SUNY_Poly_DSA598


In [4]:
!ls -a

archive			  .git				  presentation
datasets		  .gitignore			  transcribe_voice_notes.ipynb
FEVER_set_creation.ipynb  liar_gpt4omini_base_eval.ipynb  work_documents
FEVER_set_update.ipynb	  Module_2_dev.ipynb


In [5]:

def load_jsonl(file_path, encoding='utf-8'):
    """Loads a JSON Lines file into a list of Python objects."""
    data = []
    with open(file_path, 'r', encoding=encoding) as f:  # Specify encoding for safety
        for line in f:
            data.append(json.loads(line))  # Parse each line individually
    return data

In [6]:
# Data paths (replace with your actual paths if different)
#train_clf_path = f"{fever_path}tabular_clf_paper_dev_train/v1_segmented_n3461_03-29_001.csv"
#valid_clf_path = f"{fever_path}tabular_clf_paper_dev_valid/v1_segmented_n1482_03-29_001.csv"
fever_path = './datasets/FEVER/'
train_sentEx_path = f"/content/drive/MyDrive/SUNY_Poly_DSA598/datasets/FEVER/tabular_sets/tabular_sentEx_paper_dev_train/v1_segmented_n3461_03-29_001.csv"
valid_sentEx_path = f"/content/drive/MyDrive/SUNY_Poly_DSA598/datasets/FEVER/tabular_sets/tabular_sentEx_paper_dev_valid/v1_segmented_n1482_03-29_001.csv"
test_path = f"{fever_path}paper_test.jsonl"
train_path = f"{fever_path}paper_dev.jsonl"

# Load datasets
#train_clf = pd.read_csv(train_clf_path)
#valid_clf = pd.read_csv(valid_clf_path)
train_sentEx = pd.read_csv(train_sentEx_path)
valid_sentEx = pd.read_csv(valid_sentEx_path)
test_jsonl = load_jsonl(test_path)
train_jsonl = load_jsonl(train_path)

## FEVER Fine-tuning Dataset Update

### Sentence extraction from Wikipedia text, now with sentence text, page titles, sentence IDs, and entities in a list of tuples.

In [ ]:
def load_wiki_pages_minimal_memory(wiki_dir, num_files_to_load=50):
  """
  Loads Wikipedia page text from JSONL files into a dictionary (minimal memory usage).
  Loads only the 'text' field and only the first 'num_files_to_load' files.

  Args:
      wiki_dir (str): Path to the directory containing wiki-*.jsonl files.
      num_files_to_load (int): Number of wiki files to load (default: 3).

  Returns:
      dict: A dictionary where keys are page titles and values are the page text.
  """
  print(f"Attempting to load {num_files_to_load} Wikipedia pages from {wiki_dir}...")
  wiki_pages = {}
  loaded_files_count = 0
  for filename in sorted(os.listdir(wiki_dir)):
      if filename.startswith('wiki-') and filename.endswith('.jsonl'):
        filepath = os.path.join(wiki_dir, filename)
        print(f"Loading file: {filepath}")
        with open(filepath, 'r', encoding='utf-8') as f:  # Specify encoding for safety
          for line in f:
            data = json.loads(line)
            # Create a object in the dictionary for the data
            wiki_pages[data['id']] = {
              'text': data['text'],
              'lines': data['lines']
            }

        loaded_files_count += 1
      if loaded_files_count >= num_files_to_load:
            break
  return wiki_pages


wiki_pages_dir = './datasets/FEVER/wiki-pages'
num_to_load = len(os.listdir(wiki_pages_dir))
wiki_page_list_dicts = load_wiki_pages_minimal_memory(wiki_pages_dir, num_to_load)

print(f"Loaded texts from {len(wiki_page_list_dicts)} Wikipedia pages (minimal memory).")

Attempting to load 109 Wikipedia pages from ./datasets/FEVER/wiki-pages...
Loading file: ./datasets/FEVER/wiki-pages/wiki-001.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-002.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-003.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-004.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-005.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-006.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-007.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-008.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-009.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-010.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-011.jsonl
Loading file: ./datasets/FEVER/wiki-pages/wiki-012.jsonl


In [ ]:
for page, i in zip(wiki_page_list_dicts, range(1, 10)):
  print(f"Page {i}: {page}")

Page 1: 
Page 2: 1928_in_association_football
Page 3: 1986_NBA_Finals
Page 4: 1901_Villanova_Wildcats_football_team
Page 5: 1992_Northwestern_Wildcats_football_team
Page 6: 1897_Princeton_Tigers_football_team
Page 7: 1536_in_philosophy
Page 8: ...Di_terra
Page 9: 1967–68_MJHL_season


In [ ]:
import random
import requests
import re

def extract_evidence_text_debug(fever_item, wiki_page_dict, verbose=False, debug=False):
    """
    Extracts evidence sentences with debugging prints.
    """
    if verbose:
      print(f"Starting evidence extraction for claim: {fever_item['claim']}")

    if fever_item['label'] == 'NOT ENOUGH INFO':
      if verbose:
        print(f"NOT ENOUGH INFO found for claim: {fever_item['claim']}, STOP PROCESSING AND REMOVE NOT ENOUGH INFO CLAIMS")

      return [], ''

    evidence_sentences = []
    text_str = ''

    # flatten the evidence set
    evidence_set = fever_item['evidence'] # This is a list of lists of lists, where each sublist contains evidence pieces that an annotator has selected for a claim
    evidence_set = [item for sublist in evidence_set for item in sublist] # Flatten the evidence set
    # Trim the first two elements of each evidence piece (annotation_id, evidence_id)
    evidence_set = [evidence_piece[2:] for evidence_piece in evidence_set] # Trim the first two elements of each evidence piece
    if debug:
      print(f"DEBUG 0: Evidence set: {evidence_set}") # DEBUG 0
    # Convert inner lists to tuples before creating a set
    evidence_set = list(set(tuple(item) for item in evidence_set))
    # Convert back to list of lists (optional, based on your later usage)
    evidence_set = [list(item) for item in evidence_set]
    if debug:
      print(f"DEBUG 1: Length of evidence_set: {len(evidence_set)}\nSet: {evidence_set}") # DEBUG 0

    pages_loaded = set() # Set to keep track of loaded pages
    for evidence_piece in evidence_set: # Looping over evidence pieces (there will be 1 or more, these are the evidence sentences from different pages, for the same claim)
        if debug:
          print(f"DEBUG 2: Evidence piece: {evidence_piece}") # DEBUG 1
        if len(evidence_piece) == 2:  # Check if the evidence piece has 4 elements (annotation_id, evidence_id, page_title, sentence_id)
            page_title, sentence_id = evidence_piece # Unpack the evidence piece
            if debug:
              print(f"DEBUG 2.1: Processing evidence: page_title={page_title}, sentence_id={sentence_id}") # DEBUG 2
            if page_title is not None and sentence_id is not None: # Check if page_title and sentence_id are not None
                wiki_page = wiki_page_dict.get(page_title) # Retrieve the wiki page from the dictionary
                if debug:
                  print(f"DEBUG 3: Wiki page retrieved: {wiki_page is not None}") # DEBUG 3
                if wiki_page: # Check if the wiki page for these sentences
                    if debug:
                      print(f"DEBUG 4: Wiki page keys: {wiki_page.keys()}") # DEBUG 4
                    if 'lines' in wiki_page and isinstance(wiki_page['lines'], str): # Check if 'lines' key exists and is a string
                        lines_str = wiki_page['lines']
                        # Check if page has already been loaded and added to the text_str
                        if page_title not in pages_loaded:
                            pages_loaded.add(page_title)
                            text_str += "\n" + wiki_page['text']
                        elif debug:
                            print(f"DEBUG 4.1: Page {page_title} already loaded, skipping text addition.") # DEBUG 4.1
                        sentences = lines_str.strip().split('\n')
                        if debug:
                          print(f"DEBUG 5: Number of sentences found: {len(sentences)}") # DEBUG 5
                        if sentence_id < len(sentences):  # Check if sentence_id is within the range of sentences
                          for sentence, _ in zip(sentences, range(len(sentences))): # Loop over the sentences and their indices
                            if len(sentence.split('\t')) < 2 or sentence.split('\t')[1].strip() == '': #
                              if debug:
                                print(f"DEBUG 6: Skipping blank line at index {_}") # DEBUG 6
                              continue
                            if debug:
                              print(f"DEBUG 7: Line retrieved: {sentence} with ID: " +  sentence.split('\t')[0].strip()) # DEBUG 7
                            if int(sentence.split('\t')[0].strip()) == sentence_id:
                              sentence_text = sentence.split('\t')[1].strip() # Extract the text after the tab character (To skip the index at the beginning and the entities after: "0	The 1905 Tempe Normal Owls football team was an American football team that represented Tempe Normal School -LRB- later renamed Arizona State University -RRB- as an independent during the 1905 college football season .	American football	American football	Arizona State University	Arizona State University	1905 college football season	1905 college football season")
                              # Entities are everything after the second item when splitting by tab
                              entities = sentence.split('\t')[2:] # Extract the entities
                              # Remove any leading or trailing whitespace from the entities
                              entities = [entity.strip() for entity in entities if entity.strip()] # Remove empty entities
                              if debug:
                                print(f"DEBUG 8: Sentence text for index {sentence_id} / " + sentence.split('\t')[0].strip() + f" extracted: {sentence_text}") # DEBUG 8
                                print(f"DEBUG 9: Entities extracted: {entities}") # DEBUG 9
                              # Append a tuple of (sentence_text, page_title, sentence_id, entities) to the evidence_sentences list
                              evidence_sentences.append((sentence_text, page_title, sentence_id, entities))
                            #else:
                                #if verbose:
                                  #print(f"\tWarning: Sentence index does not match for page: {page_title}, sentence_id: {sentence_id}")
                        else:
                          if verbose:
                            print(f"\tWarning: Sentence index out of range for page: {page_title}, sentence_id: {sentence_id} (Number of sentences: {len(sentences)})")
                    else:
                      if verbose:
                        print(f"\tWarning: Could not retrieve page or lines for title: {page_title}")
                else:
                  if verbose:
                    print(f"\tWarning: Could not retrieve wiki page for title: {page_title}")
        else:
          if verbose:
            print(f"\tWarning: Unexpected evidence format: {evidence_piece}")
        if verbose:
          print(f"DEBUG 10: Evidence sentences for page {page_title} have been extracted.")
          print("_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")
    if verbose:
      print(f"Evidence extraction completed for claim: {fever_item['claim']}")
      print(f"Number of evidence sentences found: {len(evidence_sentences)}")
      print('--------------------------------------------------------------')
    return evidence_sentences, text_str


In [ ]:
# Test run evidence extraction
claim_item = train_jsonl[0]
evidence, full_text = extract_evidence_text_debug(claim_item, wiki_page_list_dicts, verbose=True, debug=True)

print(f"\nClaim: {claim_item['claim']}")
print("\nEvidence Sentences:")
for item in evidence:
    sentence, page_title, sentence_id, entities = item
    print(f"- {sentence}\n")
    print(f"  (Page: {page_title}, Sentence ID: {sentence_id}, Entities: {entities})")
print("\nFull Text:")
print(full_text)

Starting evidence extraction for claim: Janet Leigh was incapable of writing.
DEBUG 0: Evidence set: [['Janet_Leigh', 0], ['Janet_Leigh', 12], ['Janet_Leigh', 0], ['Janet_Leigh', 12], ['Janet_Leigh', 0], ['Janet_Leigh', 0], ['Janet_Leigh', 0]]
DEBUG 1: Length of evidence_set: 2
Set: [['Janet_Leigh', 0], ['Janet_Leigh', 12]]
DEBUG 2: Evidence piece: ['Janet_Leigh', 0]
DEBUG 2.1: Processing evidence: page_title=Janet_Leigh, sentence_id=0
DEBUG 3: Wiki page retrieved: True
DEBUG 4: Wiki page keys: dict_keys(['text', 'lines'])
DEBUG 5: Number of sentences found: 18
DEBUG 7: Line retrieved: 0	Janet Leigh -LRB- born Jeanette Helen Morrison ; July 6 , 1927 -- October 3 , 2004 -RRB- was an American actress , singer , dancer and author . with ID: 0
DEBUG 8: Sentence text for index 0 / 0 extracted: Janet Leigh -LRB- born Jeanette Helen Morrison ; July 6 , 1927 -- October 3 , 2004 -RRB- was an American actress , singer , dancer and author .
DEBUG 7: Line retrieved: 1	She is best remembered for he

### Update tabular training files

In [ ]:
# Get the claims that are SUPPORTS and REFUTES from the jsonl and the tabular data
jsonl_supports = [item for item in train_jsonl if item['label'] == 'SUPPORTS']
jsonl_refutes = [item for item in train_jsonl if item['label'] == 'REFUTES']

# Iterate through the rows of each dataframe, matching the claim text with the claim in the jsonl file, and extract the evidence sentences for that item
def update_evidence_sentences(df, jsonl_supports, jsonl_refutes, wiki_page_dict, verbose=0):
    """
    Updates the evidence sentences in the dataframe based on the JSONL data.
    """
    new_df = df.copy()  # Create a copy of the dataframe to avoid modifying the original
    for index, row in new_df.iterrows():
        claim = row['claim']
        label = row['label']
        if label == 'SUPPORTS':
            # Find the corresponding JSONL item
            jsonl_item = next((item for item in jsonl_supports if item['claim'] == claim), None)
        elif label == 'REFUTES':
            # Find the corresponding JSONL item
            jsonl_item = next((item for item in jsonl_refutes if item['claim'] == claim), None)
        else:
            # Add none if the label is not SUPPORTS or REFUTES
            if verbose:
                print(f"Adding nothing for claim: {claim} with label: {label}")
            jsonl_item = None
        if jsonl_item:
            # Extract evidence sentences and full text
            evidence_sentences, _ = extract_evidence_text_debug(jsonl_item, wiki_page_dict, verbose=verbose)
            # Update the dataframe with the evidence sentences, directly in the new_df
            new_df.at[index, 'evidence_sentences'] = evidence_sentences
        else:
            if verbose:
                print(f"NOT ENOUGH INFO found for claim: {claim}, not adding anything")
       
    return new_df

# Update the evidence sentences in the train and valid dataframes
train_sentEx_up = update_evidence_sentences(train_sentEx, jsonl_supports=jsonl_supports, jsonl_refutes=jsonl_refutes, wiki_page_dict=wiki_page_list_dicts, verbose=1)
valid_sentEx_up = update_evidence_sentences(valid_sentEx, jsonl_supports=jsonl_supports, jsonl_refutes=jsonl_refutes, wiki_page_dict=wiki_page_list_dicts, verbose=1)

In [ ]:
# Get the current time in 24 hour time, hours:minutes:seconds
current_time = datetime.datetime.now().strftime("%H:%M:%S")
# Print the seconds only
seconds = current_time.split(':')[-1]
print("22395613294602193653092643591633490623395613304603")
print(seconds)



395613294602193653092643591633490623395613304603
22


In [ ]:
# Collect metadata about the files (dataset_name  avg_evdc_len  max_evdc_len  min_evdc_len  sys_msg user_msg  sys_msg_num_tokens  sys_msg_avg_wrd_len usr_msg_num_tokens	usr_msg_avg_wrd_len)

# Helper function to tokenize text and return number of tokens and semantic complexity
def tokenize_text(text):
    """
    Tokenizes the input text and returns the number of tokens and semantic complexity.
    """
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Calculate the number of tokens
    num_tokens = len(tokens)

    # Calculate semantic complexity using a simple heuristic (e.g., average word length)
    avg_word_length = sum(len(word) for word in tokens) / num_tokens if num_tokens > 0 else 0

    return num_tokens, avg_word_length


# Create a function to collect metadata about the files
def collect_metadata(in_path, out_path, mode):
    """
    Collects metadata about the file in the input path and saves it to a CSV file in the output path.
    """
    metadata = pd.DataFrame(columns=['dataset_name', 'size', 'avg_evdc_len', 'max_evdc_len', 'min_evdc_len', 'avg_evdc_tkns', 'avg_claim_tkns', 'max_claim_tkns', 'min_claim_tkns', 'sys_msg', 'user_msg', 'sys_msg_num_tokens', 'sys_msg_avg_wrd_len', 'usr_msg_num_tokens', 'usr_msg_avg_wrd_len'])
    filename = os.path.basename(in_path)
    print(f"Processing file: {filename}")
    set_id = filename.split('_')[-1]
    if filename.endswith('.jsonl'):
        with open(in_path, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]
            ds_size = len(data)
            # Calculate average and max evidence sentence length
            evdc_counts = []
            evdc_tokens = []
            user_stub_tokens = []
            claim_tokens = []
            if mode == 'clf':
              for item in data:
                  evidence = item['messages'][1]['content'].split('\n\n')[2]
                  evdc_counts.append(len(evidence.split('\n')))
                  evdc_tokens.append(len(nltk.word_tokenize(evidence)))
                  user_stub_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[0])))
                  claim_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[1])))
            elif mode == 'sentEx':
              for item in data:
                  evdc_counts.append(len(item['messages'][2]['content'].split('\n')))
                  evdc_tokens.append(len(nltk.word_tokenize(item['messages'][2]['content'])))
                  user_stub_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[0])))
                  claim_tokens.append(len(nltk.word_tokenize(item['messages'][1]['content'].split('\n\n')[1])))

            # Calculate average, max, and min evidence sentence length
            avg_evdc_len = np.mean(evdc_counts)
            max_evdc_len = np.max(evdc_counts)
            min_evdc_len = np.min(evdc_counts)
            avg_evdc_tkns = np.mean(evdc_tokens)

            # Calculate average, max, and min claim token count
            avg_claim_len = np.mean(claim_tokens)
            max_claim_len = np.max(claim_tokens)
            min_claim_len = np.min(claim_tokens)

            item = data[0]
            sys_msg_tks, sys_msg_avg_wrd_len = tokenize_text(item['messages'][0]['content'])
            usr_msg_tks, usr_msg_avg_wrd_len = tokenize_text(item['messages'][1]['content'].split('\n\n')[0])
            # Append metadata to the DataFrame
            metadata.loc[len(metadata)] = [filename, ds_size, avg_evdc_len, max_evdc_len, min_evdc_len, avg_evdc_tkns, avg_claim_len, max_claim_len, min_claim_len, item['messages'][0]['content'], item['messages'][1]['content'].split('\n\n')[0], sys_msg_tks, sys_msg_avg_wrd_len, usr_msg_tks, usr_msg_avg_wrd_len]
    else:
        print(f"Error: {filename} is not a JSONL file.")
    # Save metadata to CSV
    save_path = out_path + f"{set_id}_metadata.csv"
    metadata.to_csv(save_path, index=False)
    print(f"Metadata saved to {save_path}")

# Collect metadata for the evidence extraction training data
sentEx_meta_paths = {
    "in_paths": [f"{fever_path}GPT_sentEx_paper_dev_train/prompt_v1_segmented_n200_03-16_001.jsonl", f"{fever_path}GPT_sentEx_paper_dev_valid/prompt_v1_segmented_n60_03-16_001.jsonl"],
    "out_paths": [f"{fever_path}GPT_meta/sentEx/train_", f"{fever_path}GPT_meta/sentEx/valid_"]
}
clf_meta_paths = {
    "in_paths": [f"{fever_path}GPT_clf_paper_dev_train/prompt_v1_segmented_n200_03-16_001.jsonl", f"{fever_path}GPT_clf_paper_dev_valid/prompt_v1_segmented_n60_03-16_001.jsonl"],
    "out_paths": [f"{fever_path}GPT_meta/clf/train_", f"{fever_path}GPT_meta/clf/valid_"]
}

for in_path, out_path in zip(sentEx_meta_paths['in_paths'], sentEx_meta_paths['out_paths']):
    collect_metadata(in_path, out_path, 'sentEx')
for in_path, out_path in zip(clf_meta_paths['in_paths'], clf_meta_paths['out_paths']):
    collect_metadata(in_path, out_path, 'clf')

Processing file: prompt_v1_segmented_n200_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/sentEx/train_001.jsonl_metadata.csv
Processing file: prompt_v1_segmented_n60_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/sentEx/valid_001.jsonl_metadata.csv
Processing file: prompt_v1_segmented_n200_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/clf/train_001.jsonl_metadata.csv
Processing file: prompt_v1_segmented_n60_03-16_001.jsonl
Metadata saved to ./datasets/FEVER/GPT_meta/clf/valid_001.jsonl_metadata.csv


In [ ]:
# Load the CSV training files and convert tokenized characters back
def replace_bracket_tokens(text):
    text = text.replace('-LRB-', '(')
    text = text.replace('-RRB-', ')')
    text = text.replace('-LSB-', '[')
    text = text.replace('-RSB-', ']')
    return text

def load_csv(filepath):
    """
    Load a CSV file and convert tokenized characters back to their original form.
    """
    df = pd.read_csv(filepath)
    # If there are three columns, replace the tokens in the first two columns
    if len(df.columns) == 3:
        df.iloc[:, 0] = df.iloc[:, 0].apply(replace_bracket_tokens)
        df.iloc[:, 1] = df.iloc[:, 1].apply(replace_bracket_tokens)
    # If there are four columns, replace the tokens in the first three columns
    elif len(df.columns) == 4:
        df.iloc[:, 0] = df.iloc[:, 0].apply(replace_bracket_tokens)
        df.iloc[:, 1] = df.iloc[:, 1].apply(replace_bracket_tokens)
        df.iloc[:, 2] = df.iloc[:, 2].apply(replace_bracket_tokens)

    return df

train_clf_path = f"{fever_path}tabular_clf_paper_dev_train/v1_segmented_n3468_03-29_001.csv"
valid_clf_path = f"{fever_path}tabular_clf_paper_dev_valid/v1_segmented_n1488_03-29_001.csv"
train_sentEx_path = f"{fever_path}tabular_sentEx_paper_dev_train/v1_segmented_n3468_03-29_001.csv"
valid_sentEx_path = f"{fever_path}tabular_sentEx_paper_dev_valid/v1_segmented_n1488_03-29_001.csv"

train_clf_df = load_csv(train_clf_path)
valid_clf_df = load_csv(valid_clf_path)
train_sentEx_df = load_csv(train_sentEx_path)
valid_sentEx_df = load_csv(valid_sentEx_path)

# Print info for each dataframe
print(f"Training data for classification:")
print(train_clf_df.info())
print(f"Training data for sentence extraction:")
print(train_sentEx_df.info())

print(f"Validation data for classification:")
print(valid_clf_df.info())
print(f"Validation data for sentence extraction:")
print(valid_sentEx_df.info())

Training data for classification:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   claim               3461 non-null   object
 1   evidence_sentences  3461 non-null   object
 2   label               3461 non-null   object
dtypes: object(3)
memory usage: 81.2+ KB
None
Training data for sentence extraction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   claim               3461 non-null   object
 1   evidence_sentences  3461 non-null   object
 2   full_text           3461 non-null   object
 3   label               3461 non-null   object
dtypes: object(4)
memory usage: 108.3+ KB
None
Validation data for classification:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 ent

In [ ]:
pd.set_option('display.max_colwidth', None)
valid_clf_df.head(20)


,claim,evidence_sentences,label
0,The Late Show with Stephen Colbert is hosted by Stephen Colbert.,"Colbert has hosted The Late Show with Stephen Colbert , a late-night television talk show on CBS , since September 8 , 2015 .",SUPPORTS
1,Annette Badland played Bad Horse in The Sparticle Mystery.,"She has played Margaret Blaine in the BBC science fiction series Doctor Who , Doomsday Dora in The Sparticle Mystery , Birdie Henshall in the drama series Cutting It , Mavis in season 6 of Skins , Ursula Crowe in children 's science fiction/fantasy series Wizards vs Aliens , and Babe Smith in soap opera EastEnders .",REFUTES
2,Good over evil is signified spiritually through Diwali.,"It is currently held every two years.\nUntil 1989, the tournament was known as CONCACAF Championship.\nThe CONCACAF Gold Cup is North America's major tournament in senior men's soccer and determines the continental champion.\nFrom 1 to 2005, nations from other confederations have regularly joined the tournament as invitees.\nIn earlier editions, the continental championship was held in different countries, but since the inception of the Gold Cup in 1991, the United States are constant hosts or co-hosts.",NOT ENOUGH INFO
3,Danger UXB is a British television series.,"Danger UXB is a 1979 British ITV television series set during the Second World War developed by John Hawkesworth and starring Anthony Andrews as Lieutenant Brian Ash , an officer in the Royal Engineers .",SUPPORTS
4,Shadowhunters was not renewed for a third season in April 2017.,"In April 2017 , it was announced that the series had been renewed for a third season of 20 episodes .",REFUTES
5,Nestor Carbonell played Richard Alpert in Lost during his presidency.,"In 2 he starred in Sparsha, which gave him breakthrough.\nSudeepa is an Indian actor and director who primarily works as leading actor role in the Kannada film industry.\nSudeep made his debut in Thayavva (1997).\nHe has also worked in Hindi, Telugu and Tamil films.\nHe is also popularly known as Kiccha Sudeepa.",NOT ENOUGH INFO
6,There is a film that is about an ex-convict titled Swordfish (film).,"The film centers on Stanley Jobson , an ex-con and computer hacker who is targeted for recruitment into a bank robbery conspiracy because of his formidable hacking skills .",SUPPORTS
7,Match Point is a brand of radios.,"Match Point is a 2005 British-Luxembourgish psychological thriller film written and directed by Woody Allen and starring Jonathan Rhys Meyers , Scarlett Johansson , Emily Mortimer , Matthew Goode , Brian Cox , and Penelope Wilton .",REFUTES
8,Girl is related to G I R L.,"Underground music is music with practices perceived as outside, or somehow opposed to, mainstream popular music culture.\nThe term has been applied to artists in styles such as psychedelia, punk, alternative rock, electronica, industrial music, and wider strains of experimental music.\nUnderground music may be perceived as expressing sincerity and creative freedom in opposition to those practices deemed formulaic or market-driven.\nUnderground styles lack the commercial success of popular music movements, and may involve the use of avant-garde or abrasive approaches.\nNotions of individuality and non-conformity are also commonly deployed.",NOT ENOUGH INFO
9,Sidse Babett Knudsen does television work.,"Knudsen achieved international recognition for her leading role as fictional Danish Prime Minister Birgitte Nyborg in the Danish TV series Borgen .\nSidse Babett Knudsen ( [ ˈsisə b̥ab̥ɛd̥ ˈkʰnusn̩ ] ; born 22 November 1968 ) is a Danish actress who works in theatre , television , and film .",SUPPORTS


In [ ]:
# Save all four dataframes back to the same directories
train_clf_df.to_csv(f"{fever_path}tabular_clf_paper_dev_train/v1_segmented_n3468_03-29_001.csv", index=False)
valid_clf_df.to_csv(f"{fever_path}tabular_clf_paper_dev_valid/v1_segmented_n1488_03-29_001.csv", index=False)
train_sentEx_df.to_csv(f"{fever_path}tabular_sentEx_paper_dev_train/v1_segmented_n3468_03-29_001.csv", index=False)
valid_sentEx_df.to_csv(f"{fever_path}tabular_sentEx_paper_dev_valid/v1_segmented_n1488_03-29_001.csv", index=False)

In [ ]:
import spacy
# Import progress bar
from tqdm import tqdm
tqdm.pandas()

def calculate_syntactic_complexity(df, verbose=False):
    """
    Calculates syntactic complexity of text in the first column of a Pandas DataFrame, per the formula from Salman et al., 2023

    Args:
        df: Pandas DataFrame with text in the first column.

    Returns:
        Pandas DataFrame with an additional 'syntactic_complexity' column.
    """

    nlp = spacy.load("en_core_web_sm")  # Load a suitable spaCy model

    tw = 0.07  # Token weight
    vw = 0.3   # Verb weight
    cw = 0.4   # Conjunction weight

    sc_scores = []

    # Wrap the iterable with tqdm for progress bar
    for text in tqdm(df.iloc[:, 0], desc="Calculating syntactic complexity"):
        doc = nlp(text)
        tkn = len(doc)
        vrb = 0
        cnj = 0

        for token in doc:
            if token.pos_ == "VERB":
                vrb += 1
            elif token.pos_ in {"CONJ", "CCONJ", "SCONJ"}:  # Use a set for conjunction categories
                cnj += 1

        sc = tkn * tw + vrb * vw + cnj * cw
        sc_scores.append(sc)

    df['syntactic_complexity'] = sc_scores  # Add results to DataFrame
    return df

#train_clf_df = calculate_syntactic_complexity(train_clf_df, True)
valid_clf_df = calculate_syntactic_complexity(valid_clf_df)
train_sentEx_df = calculate_syntactic_complexity(train_sentEx_df)
valid_sentEx_df = calculate_syntactic_complexity(valid_sentEx_df)

Calculating syntactic complexity: 100%|██████████| 1482/1482 [00:11<00:00, 126.35it/s]


In [ ]:
# Print info for each dataframe
print(f"Training data for classification:")
print(train_clf_df.info())
print(f"Training data for sentence extraction:")
print(train_sentEx_df.info())

print(f"Validation data for classification:")
print(valid_clf_df.info())
print(f"Validation data for sentence extraction:")
print(valid_sentEx_df.info())

Training data for classification:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   claim                 3461 non-null   object 
 1   evidence_sentences    3461 non-null   object 
 2   label                 3461 non-null   object 
 3   syntactic_complexity  3461 non-null   float64
dtypes: float64(1), object(3)
memory usage: 108.3+ KB
None
Training data for sentence extraction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   claim                 3461 non-null   object 
 1   evidence_sentences    3461 non-null   object 
 2   full_text             3461 non-null   object 
 3   label                 3461 non-null   object 
 4   syntactic_complexity  3461 non-null   f

In [ ]:
# Drop 4 samples from valid_sentEx_df randomly
valid_sentEx_df = valid_sentEx_df.drop(valid_sentEx_df.sample(n=4).index)

In [ ]:
valid_sentEx_df.reset_index(drop=True, inplace=True)